In [ ]:
# required modules 
import random 
import json 
import pickle 
import numpy as np 
import nltk 
from keras.models import load_model 
from nltk.stem import WordNetLemmatizer 
from datetime import datetime 
import tkinter as tk
from tkinter import scrolledtext
import requests
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
api_key = 'aa211acf060448ee5b6d2cced816d68f'
city = 'Allahabad'  # Replace 'London' with the city you want to check

In [ ]:
lemmatizer = WordNetLemmatizer() 

# loading the files we made previously 
intents = json.loads(open("intents/intents.json").read()) 
words = pickle.load(open('words.pkl', 'rb')) 
#print(words) 
classes = pickle.load(open('classes.pkl', 'rb')) 
model = load_model('chatbotmodel.h5') 

This cell is get wether condition from the API of openweathermap of Allahabad

In [ ]:
def get_weather():
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx errors
        data = response.json()
        if data["cod"] == 200:
            weather = data["weather"][0]["description"]
            return weather
        else:
            print(f"Could not get weather data for {city}")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

This cell finds the output of any arithematic equation  

In [ ]:
import ast

def calculate_expression(expression):
    try:
        # Parse the expression
        parsed_expr = ast.parse(expression, mode='eval')
        # Evaluate the parsed expression
        result = eval(compile(parsed_expr, '<string>', 'eval'))
        return result
    except Exception as e:
        return f"Error: {e}"

This cell recognizes the keywords related to real time or calculation and return the output

In [ ]:
def genral_question(intent, s):
    now = datetime.now()
    #print(intent)
    if(intent == 'date'):
        return now.date()
    
    elif(intent == 'time'):
        return now.strftime("%H:%M:%S")
    
    elif(intent == 'weekday'):
        return days[now.weekday()]

    elif(intent == 'weather'):
        return get_weather()
    
    elif(intent == 'calculation'):
        for i in range(len(s)):
            if(s[i] == '-' or s[i].isnumeric()):
                return calculate_expression(s[i:])
    else:
        return " "
    

This cell coverts the message to the updated message which does not contains any dulicates and not having the grammar involved

In [ ]:
def clean_up_sentences(sentence): 
    sentence_words = nltk.word_tokenize(sentence) 
    sentence_words = [lemmatizer.lemmatize(word) 
                      for word in sentence_words] 
    return sentence_words 

This cell finds the words present in the message and finds the words which are present in the intents and mark it as 1

In [ ]:
def bagw(sentence): 
    sentence = sentence.lower()
    # separate out words from the input sentence 
    sentence_words = clean_up_sentences(sentence) 
    bag = [0]*len(words) 
    for w in sentence_words: 
        for i, word in enumerate(words): 
  
            # check whether the word 
            # is present in the input as well 
            if word == w: 
  
                # as the list of words 
                # created earlier. 
                bag[i] = 1
  
    # return a numpy array 
    return np.array(bag) 

This cell predicts the class for the message given by the user 

In [ ]:
def predict_class(sentence): 
    bow = bagw(sentence) 
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) 
               if r > ERROR_THRESHOLD] 
    results.sort(key=lambda x: x[1], reverse=True) 
    print(results)
    return_list = [] 
    for r in results: 
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])}) 
    
    return return_list 

This cell return the responce to the user after finding the best suitable answer from the model

In [ ]:
def get_response(intents_list, intents_json): 
    tag = intents_list[0]['intent'] 
    list_of_intents = intents_json['intents'] 
    result = "I can't undersatnd!" 
    for i in list_of_intents: 
        if i['tag'] == tag: 
            result = random.choice(i['responses']) 
            break
    return result 

print("Chatbot is up!") 

In [ ]:
def send_message(event=None):
    message = input_field.get("1.0", tk.END).strip()
    if message.lower() == 'exit':
        root.destroy()
        return
    input_field.delete("1.0", tk.END)
    chat_log.insert(tk.END, f"You : {message}\n", "user")
    ints = predict_class(message)
    res = get_response(ints, intents)
    res += str(genral_question(ints[0]['intent'], message))
    chat_log.insert(tk.END, f"Bot : {res}\n", "bot")
    chat_log.see(tk.END)

root = tk.Tk()
root.title("Chatbot")
root.configure(bg='#303030')  # Set background color to a lighter shade of black

# Create a scrolled text widget for chat log
chat_log = scrolledtext.ScrolledText(root, width=60, height=20, wrap=tk.WORD, bg='#303030', fg='white', insertbackground='white')
chat_log.pack()

# Add tags for user and bot messages
chat_log.tag_configure("user", foreground="orange")
chat_log.tag_configure("bot", foreground="green")

# Create a frame to contain the input field and send button
input_frame = tk.Frame(root, bg='#404040')
input_frame.pack(pady=10, fill=tk.X)

# Create an input field with a lighter shade, rounded corners, and centered alignment
input_field = tk.Text(input_frame, height=1, width=50, bg='#404040', fg='white', insertbackground='white', relief=tk.FLAT, wrap=tk.WORD)
input_field.pack(side=tk.LEFT)
input_field.config(font=("Arial", 12), spacing1=10)
input_field.bind('<Return>', send_message)

# Create a send button
send_button = tk.Button(input_frame, text="Send", command=send_message)
send_button.pack(side=tk.LEFT)

root.mainloop()

In [ ]:
# while True:
#     message = input("")
#     if(message == 'exit'):
#         break 
#     ints = predict_class(message) 
#     res = get_response(ints, intents) 
#     res += str(genral_question(ints[0]['intent'], message)) 
#     print(res) 